In [32]:
import pandas as pd
import numpy as np

np.random.seed(121)

import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers

In [33]:
df_train = pd.read_csv('train.csv')


In [34]:
df_train.head() 

label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0         0         0         0   
3       0  ...         0         0         0         0         0         0   
4       0  ...         0         0         0         0         0         0   

   pixel780  pixel781  pixel782  pixel783  
0         0         0         0         0  
1         0         0         0         0  
2         0         0         0         0  
3         0         0         0         0  
4         0         0         0         0  

[5 rows x 785 columns]

In [35]:
df_train.tail() 

label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
41995      0       0       0       0       0       0       0       0       0   
41996      1       0       0       0       0       0       0       0       0   
41997      7       0       0       0       0       0       0       0       0   
41998      6       0       0       0       0       0       0       0       0   
41999      9       0       0       0       0       0       0       0       0   

       pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  \
41995       0  ...         0         0         0         0         0   
41996       0  ...         0         0         0         0         0   
41997       0  ...         0         0         0         0         0   
41998       0  ...         0         0         0         0         0   
41999       0  ...         0         0         0         0         0   

       pixel779  pixel780  pixel781  pixel782  pixel783  
41995         0         0         0         0         0  
41996         0         0         0         0         0  
41997         0         0         0         0         0  
41998         0         0         0         0         0  
41999         0         0         0         0         0  

[5 rows x 785 columns]

In [36]:
print(df_train.shape)

(42000, 785)


In [37]:
df_features = df_train.iloc[:, 1:785]
df_label = df_train.iloc[:, 0]


In [38]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(df_features, df_label, 
                                                test_size = 0.2,
                                                random_state = 121)


In [39]:
# Feature Normalization 
X_train = X_train.astype('float32')
X_cv= X_cv.astype('float32')

X_train /= 255
X_cv /= 255


# Convert labels to One Hot Encoded
# https://keras.io/utils/
num_digits = 10
y_train = keras.utils.to_categorical(y_train, num_digits)
y_cv = keras.utils.to_categorical(y_cv, num_digits)


In [40]:
# Printing 2 examples of labels after conversion
print(y_train[0]) # 2
print(y_train[3]) # 7

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [41]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
n_hidden_5 = 150
num_digits = 10

In [42]:
Inp = Input(shape=(784,))
x1 = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x2 = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x1)
x3 = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x2)
x4 = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x3)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x4)

In [43]:
# Our model would have '6' layers - input layer, 4 hidden layer and 1 output layer
model = Model(Inp, output)
model.summary() # We have 297,910 parameters to estimate

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 784)               0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
Hidden_Layer_4 (Dense)       (None, 200)               20200     
_________________________________________________________________
Output_Layer (Dense)         (None, 10)                2010      
Total params: 297,910
Trainable params: 297,910
Non-trainable params: 0
_____________________________________________________

In [44]:
# Insert Hyperparameters
learning_rate = 0.1
training_epochs = 20
batch_size = 100
#sgd = optimizers.SGD(lr=learning_rate)
sgd= keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [45]:
# We rely on the  Stochastic Gradient Descent as our optimizing methodology
# https://keras.io/losses/
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [46]:
# https://stackoverflow.com/questions/47902295/what-is-the-use-of-verbose-in-keras-while-validating-the-model
history1 = model.fit(X_train, y_train,
                     batch_size = batch_size,
                     epochs = training_epochs,
                     verbose = 2,
                     validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
 - 2s - loss: 0.3433 - accuracy: 0.8986 - val_loss: 0.1691 - val_accuracy: 0.9514
Epoch 2/20
 - 2s - loss: 0.1229 - accuracy: 0.9630 - val_loss: 0.1245 - val_accuracy: 0.9620
Epoch 3/20
 - 2s - loss: 0.0833 - accuracy: 0.9734 - val_loss: 0.1151 - val_accuracy: 0.9660
Epoch 4/20
 - 2s - loss: 0.0600 - accuracy: 0.9806 - val_loss: 0.1400 - val_accuracy: 0.9629
Epoch 5/20
 - 2s - loss: 0.0427 - accuracy: 0.9865 - val_loss: 0.1094 - val_accuracy: 0.9688
Epoch 6/20
 - 2s - loss: 0.0372 - accuracy: 0.9883 - val_loss: 0.0985 - val_accuracy: 0.9719
Epoch 7/20
 - 2s - loss: 0.0289 - accuracy: 0.9907 - val_loss: 0.1176 - val_accuracy: 0.9706
Epoch 8/20
 - 2s - loss: 0.0233 - accuracy: 0.9921 - val_loss: 0.1111 - val_accuracy: 0.9719
Epoch 9/20
 - 2s - loss: 0.0219 - accuracy: 0.9932 - val_loss: 0.1175 - val_accuracy: 0.9713
Epoch 10/20
 - 2s - loss: 0.0259 - accuracy: 0.9916 - val_loss: 0.1368 - val_accuracy: 0.9668
Epoch 11/20
 - 2s - 